In [ ]:
!sudo pip install git+https://github.com/rcmalli/keras-vggface.git

In [ ]:
pip install Keras-Applications

In [ ]:
import os
import pandas as pd 
import numpy as np
import imageio
import seaborn as sns
import random
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from collections import defaultdict
import cv2 
from tqdm import tqdm
import pickle 

import tensorflow
from tensorflow import keras 
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from tensorflow.keras import optimizers
from keras import preprocessing 
from keras import applications 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras_vggface.vggface import VGGFace

In [ ]:
!ls

content  data_3000  data_3000_comprred.zip  sample_data  vg_face_model.zip


In [ ]:
!unzip /content/vg_face_model.zip

In [ ]:
model = keras.models.load_model('/content/content/vg_face_model')

In [ ]:
!unzip /content/data_3000_comprred.zip

In [ ]:
!ls data_3000/train/

anger  disgust	fear  happiness  neutrality  sadness  surprise


In [ ]:
def find_poor_images(model, emotion="anger", data="../raw_data/data_100/test", top_n=3):
    """
    Input: model, class, data folder location and top value for prediction to be in
    Returns: image names of poorly classified in given folder
    """
    poor_label = []
    true = classes.index(emotion)
    for im in tqdm(os.listdir(f'{data}/{emotion}/')):
        image = cv2.cvtColor(imageio.imread(f'{data}/{emotion}/{im}'),cv2.COLOR_GRAY2RGB)
        im244 = np.expand_dims(tensorflow.image.resize(image, [224, 224]),axis=0)/255.0
        pred = model.predict(im244)
        top_values = pred.argsort()[0][-top_n:]
        if true not in top_values:
            poor_label.append(im)
    return poor_label

In [ ]:
classes=['surprise',
        "fear",
        "anger",
        "neutrality",
        "sadness",
        "happiness"]
dict_poor_3000_train = {}
data_location = "/content/data_3000/train"

for emotion in classes:
    poor_label = find_poor_images(model, emotion=emotion, data=data_location)
    dict_poor_3000_train[emotion] = poor_label

In [ ]:
dict_poor_3000_train.keys()

dict_keys(['surprise', 'fear', 'anger', 'neutrality', 'sadness', 'happiness'])

In [ ]:
for k, v in dict_poor_3000_train.items():
    print(k, len(v))

surprise 172
fear 437
anger 1634
neutrality 1439
sadness 1305
happiness 1991


In [ ]:
with open("dict_poor_3000_train", 'wb') as pickle_file:
    pickle.dump(dict_poor_3000_train, pickle_file)

In [ ]:
print(f"##########{data_location}###########")

for dir_ in os.listdir(data_location):
    count = 0
    for f in os.listdir(f"{data_location}/{dir_}/"):
        count += 1
    print(f"{dir_}:   \t {count} images")

##########/content/data_3000/train###########
neutrality:   	 2700 images
disgust:   	 2700 images
anger:   	 2700 images
surprise:   	 2700 images
happiness:   	 2700 images
sadness:   	 2700 images
fear:   	 2700 images


In [ ]:
for k, v in dict_poor_3000_train.items():
    for pic in v:
        os.remove(f"{data_location}/{k}/{pic}")

In [ ]:
print(f"##########{data_location}###########")

for dir_ in os.listdir(data_location):
    count = 0
    for f in os.listdir(f"{data_location}/{dir_}/"):
        count += 1
    print(f"{dir_}:   \t {count} images")

##########/content/data_3000/train###########
neutrality:   	 1261 images
disgust:   	 2700 images
anger:   	 1066 images
surprise:   	 2528 images
happiness:   	 709 images
sadness:   	 1395 images
fear:   	 2263 images


In [ ]:
!ls

content    data_3000_comprred.zip  sample_data
data_3000  dict_poor_3000_train    vg_face_model.zip


In [ ]:
from google.colab import files
!zip -r new_3000.zip data_3000

files.download("new_3000.zip")

In [ ]:
def split_data(data_dir, train, subsample=True, samplesize=700):
    """
    Inputs:The current data dir for a given class, the destination train, val and test dir. 
           PLus the ratio of train size.
    Output: Copies of the the images in the correct destination file locations.
    """
    #make list of of file names
    current_data = []
    for item in os.listdir(data_dir):
        if os.path.getsize(data_dir+item) >0:
            current_data.append(item)
        else:
            print(item, "Source is empty!")

    shuff = random.sample(current_data, samplesize)
    
    #copy files into correct locations
    for item in shuff:
        copyfile(data_dir + item, train + item)


In [ ]:
!ls content/new_700/

In [ ]:
classes=["disgust",
        "neutrality"]
#os.mkdir('content/new_700/train/')


for class_ in classes:
    os.mkdir(f'content/new_700/train/{class_}')

In [ ]:
classes=['surprise',
        "fear",
        "anger",
         "disgust",
        "neutrality",
        "sadness",
        "happiness"]
for emotion in tqdm(classes):
    split_data(f"data_3000/train/{emotion}/", f"content/new_700/train/{emotion}/", subsample=True, samplesize=700)


100%|██████████| 7/7 [00:00<00:00, 19.47it/s]


In [ ]:
print(f"##########{data_location}###########")

for dir_ in os.listdir(data_location):
    count = 0
    for f in os.listdir(f"{data_location}/{dir_}/"):
        count += 1
    print(f"{dir_}:   \t {count} images")

In [ ]:
train_dir = "/content/content/new_700/train"
train_datagen = ImageDataGenerator(rescale=1.0/255., 
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest'
                                    )


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=32,
                                                    classes=['surprise',
                                                            "fear",
                                                            "disgust",
                                                            "anger",
                                                            "neutrality",
                                                            "sadness",
                                                            "happiness"],
                                                    class_mode="categorical",
                                                    color_mode="rgb",
                                                    target_size=(224, 224))
val_dir= "data_3000/val/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)

validation_generator = validation_datagen.flow_from_directory(val_dir,
                                                              batch_size=32,
                                                              classes=['surprise',
                                                                        "fear",
                                                                        "disgust",
                                                                        "anger",
                                                                        "neutrality",
                                                                        "sadness",
                                                                        "happiness"],
                                                              class_mode="categorical",
                                                              color_mode="rgb",
                                                              target_size=(224, 224)
                                                             
                                                             )

Found 6892 images belonging to 7 classes.
Found 1050 images belonging to 7 classes.


In [ ]:
keras.backend.clear_session()
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=20, verbose=1, restore_best_weights=True)
history = model.fit(train_generator,
                            epochs=100,verbose=1,
                            validation_data=validation_generator,
                            callbacks=[es])

Epoch 1/100
216/216 [==============================] - 113s 521ms/step - loss: 1.8252 - accuracy: 0.2462 - val_loss: 1.7825 - val_accuracy: 0.2295
Epoch 2/100
216/216 [==============================] - 108s 499ms/step - loss: 1.8063 - accuracy: 0.2564 - val_loss: 1.7596 - val_accuracy: 0.2476
Epoch 3/100
216/216 [==============================] - 107s 496ms/step - loss: 6.8618 - accuracy: 0.2784 - val_loss: 2.0328 - val_accuracy: 0.1429
Epoch 4/100
216/216 [==============================] - 107s 494ms/step - loss: 1.8633 - accuracy: 0.2192 - val_loss: 1.9861 - val_accuracy: 0.1438
Epoch 5/100
216/216 [==============================] - 107s 495ms/step - loss: 1.8562 - accuracy: 0.2294 - val_loss: 1.9707 - val_accuracy: 0.1467
Epoch 6/100
216/216 [==============================] - 107s 494ms/step - loss: 1.8537 - accuracy: 0.2323 - val_loss: 1.9792 - val_accuracy: 0.1476
Epoch 7/100
216/216 [==============================] - 107s 493ms/step - loss: 1.8502 - accuracy: 0.2327 - val_loss: 2

KeyboardInterrupt: ignored